In [1]:
import os
import numpy as np
import pandas as pd
import cv2
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, UpSampling2D, concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.regularizers import l1_l2
from sklearn.model_selection import train_test_split

# TensorFlow GPU configuration
physical_devices = tf.config.list_physical_devices('GPU')
if len(physical_devices) > 0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
    print("Using GPU:", physical_devices[0])
else:
    print("No GPU found, using CPU")

# Set up paths and categories
base_data_dir = '/blue/srampazzi/vi.gade/cov/covid'
categories = ['COVID', 'Lung_Opacity', 'Normal', 'Viral_Pneumonia']

# Load metadata and preprocess
all_metadata = pd.DataFrame()
for category in categories:
    path = f'{base_data_dir}/{category}.metadata.xlsx'
    df = pd.read_excel(path, usecols=['FILE NAME', 'FORMAT', 'SIZE'])
    df['label'] = category
    df['image_path'] = df['FILE NAME'].apply(lambda x: f'{base_data_dir}/{category}/images/{x}.png')
    df['mask_path'] = df['FILE NAME'].apply(lambda x: f'{base_data_dir}/{category}/masks/{x}.png')
    df = df[df['image_path'].apply(os.path.exists)]  # Ensure file exists
    all_metadata = pd.concat([all_metadata, df], ignore_index=True)

# Data splitting
train_df, temp_df = train_test_split(all_metadata, test_size=0.4, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

# Data augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Data generator function with data augmentation and masks
def data_generator_segmentation(df, batch_size=32):
    while True:
        batch_paths = df.sample(n=batch_size)
        batch_input = []
        batch_output = []
        for _, row in batch_paths.iterrows():
            image = cv2.imread(row['image_path'], cv2.IMREAD_GRAYSCALE)  # Read as grayscale image
            mask = cv2.imread(row['mask_path'], cv2.IMREAD_GRAYSCALE)
            if mask is not None:
                image = cv2.resize(image, (256, 256))
                mask = cv2.resize(mask, (256, 256))
                image = image / 255.0
                mask = mask / 255.0
                mask = np.round(mask).astype(np.float32)
                image = np.stack((image,)*3, axis=-1)  # Replicate grayscale channel to 3 channels
                image = datagen.random_transform(image)  # Apply random transform after replication
                mask = np.expand_dims(mask, axis=-1)
                batch_input.append(image)
                batch_output.append(mask)
        batch_x = np.array(batch_input)
        batch_y = np.array(batch_output)
        yield batch_x, batch_y

# Setup data generators with data augmentation and masks
train_gen_segmentation = data_generator_segmentation(train_df, batch_size=32)
val_gen_segmentation = data_generator_segmentation(val_df, batch_size=32)
test_gen_segmentation = data_generator_segmentation(test_df, batch_size=32)

# Improved U-Net model with ResNet50 encoder and U-Net decoder maintaining 256x256 resolution
def build_unet_resnet50(input_shape):
    inputs = Input(shape=input_shape)

    # Encoder: ResNet50
    base_model = ResNet50(weights='imagenet', include_top=False, input_tensor=inputs)
    skip_connection_names = ['conv1_relu', 'conv2_block3_out', 'conv3_block4_out', 'conv4_block6_out']
    layers = [base_model.get_layer(name).output for name in skip_connection_names]

    # Decoder
    x = base_model.output
    num_filters = 256
    regularizer = l1_l2(l1=0.01, l2=0.1)
    for i in range(len(layers)):
        x = UpSampling2D()(x)
        x = Conv2D(num_filters // (2 ** i), (3, 3), activation='relu', padding='same', kernel_regularizer=regularizer)(x)
        x = concatenate([x, layers[-(i + 1)]])
        x = Conv2D(num_filters // (2 ** i), (3, 3), activation='relu', padding='same', kernel_regularizer=regularizer)(x)

    x = UpSampling2D(size=(2, 2))(x)
    outputs = Conv2D(1, (1, 1), activation='sigmoid')(x)
    model = Model(inputs, outputs)
    return model

input_shape = (256, 256, 3)  # Change input shape to 3 channels
unet_resnet50 = build_unet_resnet50(input_shape)

def dice_loss(y_true, y_pred):
    y_true_f = tf.keras.backend.flatten(y_true)
    y_pred_f = tf.keras.backend.flatten(y_pred)
    intersection = tf.keras.backend.sum(y_true_f * y_pred_f)
    return 1 - (2. * intersection + 1) / (tf.keras.backend.sum(y_true_f) + tf.keras.backend.sum(y_pred_f) + 1)

def combined_loss(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)
    bce = tf.keras.losses.binary_crossentropy(y_true, y_pred)
    dice = dice_loss(y_true, y_pred)
    return bce + dice

unet_resnet50.compile(optimizer=Adam(learning_rate=0.0001), loss=combined_loss, metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=1e-6)
model_checkpoint = ModelCheckpoint('best_modellatestt2.h5', save_best_only=True, monitor='val_loss')

history_segmentation = unet_resnet50.fit(
    train_gen_segmentation,
    steps_per_epoch=len(train_df) // 32,
    validation_data=val_gen_segmentation,
    validation_steps=len(val_df) // 32,
    epochs=50,
    callbacks=[early_stopping, reduce_lr, model_checkpoint]
)

test_results_segmentation = unet_resnet50.evaluate(test_gen_segmentation, steps=len(test_df) // 32)
print(f"Test accuracy with segmentation model: {test_results_segmentation[1]*100:.2f}%")

unet_resnet50.save('final_modellatestt2')


Using GPU: PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


2024-06-20 18:03:23.614323: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-20 18:03:24.328967: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 78902 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-80GB, pci bus id: 0000:b7:00.0, compute capability: 8.0


Epoch 1/50


2024-06-20 18:03:30.084749: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8201


205/205 [==============================] - ETA: 0s - loss: 420.9431 - accuracy: 0.8475

/apps/tensorflow/2.7.0/lib/python3.9/site-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


205/205 [==============================] - 121s 552ms/step - loss: 420.9431 - accuracy: 0.8475 - val_loss: 79.2858 - val_accuracy: 0.7720 - lr: 1.0000e-04
Epoch 2/50
205/205 [==============================] - 112s 546ms/step - loss: 35.9850 - accuracy: 0.8804 - val_loss: 15.0321 - val_accuracy: 0.7716 - lr: 1.0000e-04
Epoch 3/50
205/205 [==============================] - 113s 552ms/step - loss: 7.9157 - accuracy: 0.8861 - val_loss: 5.0120 - val_accuracy: 0.8145 - lr: 1.0000e-04
Epoch 4/50
205/205 [==============================] - 112s 549ms/step - loss: 3.2017 - accuracy: 0.8889 - val_loss: 2.4932 - val_accuracy: 0.8550 - lr: 1.0000e-04
Epoch 5/50
205/205 [==============================] - 114s 556ms/step - loss: 2.0635 - accuracy: 0.8919 - val_loss: 1.9629 - val_accuracy: 0.8708 - lr: 1.0000e-04
Epoch 6/50
205/205 [==============================] - 110s 539ms/step - loss: 1.8478 - accuracy: 0.8942 - val_loss: 2.0330 - val_accuracy: 0.8303 - lr: 1.0000e-04
Epoch 7/50
205/205 [========

2024-06-20 19:35:24.546209: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: final_modellatestt2/assets


/apps/tensorflow/2.7.0/lib/python3.9/site-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)
/apps/tensorflow/2.7.0/lib/python3.9/site-packages/keras/saving/saved_model/layer_serialization.py:112: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  return generic_utils.serialize_keras_object(obj)


In [1]:
import os
import numpy as np
import pandas as pd
import cv2
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, UpSampling2D, concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.regularizers import l1_l2
from sklearn.model_selection import train_test_split

# TensorFlow GPU configuration
physical_devices = tf.config.list_physical_devices('GPU')
if len(physical_devices) > 0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
    print("Using GPU:", physical_devices[0])
else:
    print("No GPU found, using CPU")

# Set up paths and categories
base_data_dir = '/blue/srampazzi/vi.gade/cov/covid'
categories = ['COVID', 'Lung_Opacity', 'Normal', 'Viral_Pneumonia']

# Load metadata and preprocess
all_metadata = pd.DataFrame()
for category in categories:
    path = f'{base_data_dir}/{category}.metadata.xlsx'
    df = pd.read_excel(path, usecols=['FILE NAME', 'FORMAT', 'SIZE'])
    df['label'] = category
    df['image_path'] = df['FILE NAME'].apply(lambda x: f'{base_data_dir}/{category}/images/{x}.png')
    df['mask_path'] = df['FILE NAME'].apply(lambda x: f'{base_data_dir}/{category}/masks/{x}.png')
    df = df[df['image_path'].apply(os.path.exists)]  # Ensure file exists
    all_metadata = pd.concat([all_metadata, df], ignore_index=True)

# Data splitting
train_df, temp_df = train_test_split(all_metadata, test_size=0.4, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

# Data augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Data generator function with data augmentation and masks
def data_generator_segmentation(df, batch_size=32):
    while True:
        batch_paths = df.sample(n=batch_size)
        batch_input = []
        batch_output = []
        for _, row in batch_paths.iterrows():
            image = cv2.imread(row['image_path'], cv2.IMREAD_COLOR)
            mask = cv2.imread(row['mask_path'], cv2.IMREAD_GRAYSCALE)  # Load the mask
            if mask is not None:  # Ensure mask is loaded correctly
                image = cv2.resize(image, (256, 256))
                mask = cv2.resize(mask, (256, 256))  # Keep mask size to 256x256
                image = image / 255.0
                mask = mask / 255.0
                mask = np.round(mask).astype(np.float32)  # Ensure mask is binary and float32
                image = datagen.random_transform(image)
                mask = np.expand_dims(mask, axis=-1)  # Expand mask dimensions to (256, 256, 1)
                batch_input.append(image)
                batch_output.append(mask)  # Append mask directly
        batch_x = np.array(batch_input)
        batch_y = np.array(batch_output)
        yield batch_x, batch_y

# Setup data generators with data augmentation and masks
train_gen_segmentation = data_generator_segmentation(train_df, batch_size=32)
val_gen_segmentation = data_generator_segmentation(val_df, batch_size=32)
test_gen_segmentation = data_generator_segmentation(test_df, batch_size=32)

# Improved U-Net model with ResNet50 encoder and U-Net decoder maintaining 256x256 resolution
def build_unet_resnet50(input_shape):
    inputs = Input(shape=input_shape)

    # Encoder: ResNet50
    base_model = ResNet50(weights='imagenet', include_top=False, input_tensor=inputs)
    skip_connection_names = ['conv1_relu', 'conv2_block3_out', 'conv3_block4_out', 'conv4_block6_out']
    layers = [base_model.get_layer(name).output for name in skip_connection_names]

    # Decoder
    x = base_model.output
    num_filters = 256
    regularizer = l1_l2(l1=0.01, l2=0.01)
    for i in range(len(layers)):
        x = UpSampling2D()(x)
        x = Conv2D(num_filters // (2 ** i), (3, 3), activation='relu', padding='same', kernel_regularizer=regularizer)(x)
        x = concatenate([x, layers[-(i + 1)]])
        x = Conv2D(num_filters // (2 ** i), (3, 3), activation='relu', padding='same', kernel_regularizer=regularizer)(x)

    # Ensure final output matches the original input size
    x = UpSampling2D(size=(2, 2))(x)  # Up-sample to 256x256
    outputs = Conv2D(1, (1, 1), activation='sigmoid')(x)  # Use sigmoid activation for binary segmentation
    model = Model(inputs, outputs)
    return model

input_shape = (256, 256, 3)
unet_resnet50 = build_unet_resnet50(input_shape)

# Define the combined loss function
def dice_loss(y_true, y_pred):
    y_true_f = tf.keras.backend.flatten(y_true)
    y_pred_f = tf.keras.backend.flatten(y_pred)
    intersection = tf.keras.backend.sum(y_true_f * y_pred_f)
    return 1 - (2. * intersection + 1) / (tf.keras.backend.sum(y_true_f) + tf.keras.backend.sum(y_pred_f) + 1)

def combined_loss(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float32)  # Ensure y_true is float32
    y_pred = tf.cast(y_pred, tf.float32)  # Ensure y_pred is float32
    bce = tf.keras.losses.binary_crossentropy(y_true, y_pred)
    dice = dice_loss(y_true, y_pred)
    return bce + dice

unet_resnet50.compile(optimizer=Adam(learning_rate=0.0001), loss=combined_loss, metrics=['accuracy'])

# Callbacks for early stopping and learning rate reduction
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=1e-6)

# Model training with data augmentation and regularization
history_segmentation = unet_resnet50.fit(
    train_gen_segmentation,
    steps_per_epoch=len(train_df) // 32,
    validation_data=val_gen_segmentation,
    validation_steps=len(val_df) // 32,
    epochs=50,
    callbacks=[early_stopping, reduce_lr]
)

# Evaluate the model on the test set with augmentation
test_results_segmentation = unet_resnet50.evaluate(test_gen_segmentation, steps=len(test_df) // 32)
print(f"Test accuracy with segmentation model: {test_results_segmentation[1]*100:.2f}%")

# Save the trained model in TensorFlow SavedModel format
model_tf_save_path = 'unet_resnet50_segmentation_model'
unet_resnet50.save(model_tf_save_path, save_format='tf')
print(f"Model saved to {model_tf_save_path} directory")

Using GPU: PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


2024-06-20 19:42:35.900989: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-20 19:42:36.784121: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 16659 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-80GB, pci bus id: 0000:b7:00.0, compute capability: 8.0


Epoch 1/50


2024-06-20 19:42:42.999124: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8201


205/205 [==============================] - 122s 553ms/step - loss: 366.3969 - accuracy: 0.8439 - val_loss: 63.3599 - val_accuracy: 0.7688 - lr: 1.0000e-04
Epoch 2/50
205/205 [==============================] - 113s 552ms/step - loss: 27.9995 - accuracy: 0.8786 - val_loss: 12.0669 - val_accuracy: 0.7734 - lr: 1.0000e-04
Epoch 3/50
205/205 [==============================] - 115s 565ms/step - loss: 6.6921 - accuracy: 0.8855 - val_loss: 4.8252 - val_accuracy: 0.8050 - lr: 1.0000e-04
Epoch 4/50
205/205 [==============================] - 114s 557ms/step - loss: 3.3162 - accuracy: 0.8896 - val_loss: 3.1510 - val_accuracy: 0.8144 - lr: 1.0000e-04
Epoch 5/50
205/205 [==============================] - 113s 553ms/step - loss: 2.4273 - accuracy: 0.8914 - val_loss: 2.4743 - val_accuracy: 0.8405 - lr: 1.0000e-04
Epoch 6/50
205/205 [==============================] - 114s 558ms/step - loss: 2.1145 - accuracy: 0.8921 - val_loss: 2.1027 - val_accuracy: 0.8749 - lr: 1.0000e-04
Epoch 7/50
205/205 [========

2024-06-20 21:19:11.948174: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: unet_resnet50_segmentation_model/assets
Model saved to unet_resnet50_segmentation_model directory


/apps/tensorflow/2.7.0/lib/python3.9/site-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)
/apps/tensorflow/2.7.0/lib/python3.9/site-packages/keras/saving/saved_model/layer_serialization.py:112: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  return generic_utils.serialize_keras_object(obj)


In [1]:
import os
import numpy as np
import pandas as pd
import cv2
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, UpSampling2D, concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.regularizers import l1_l2
from sklearn.model_selection import train_test_split

# TensorFlow GPU configuration
physical_devices = tf.config.list_physical_devices('GPU')
if len(physical_devices) > 0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
    print("Using GPU:", physical_devices[0])
else:
    print("No GPU found, using CPU")

# Set up paths and categories
base_data_dir = '/blue/srampazzi/vi.gade/cov/covid'
categories = ['COVID', 'Lung_Opacity', 'Normal', 'Viral_Pneumonia']

# Load metadata and preprocess
all_metadata = pd.DataFrame()
for category in categories:
    path = f'{base_data_dir}/{category}.metadata.xlsx'
    df = pd.read_excel(path, usecols=['FILE NAME', 'FORMAT', 'SIZE'])
    df['label'] = category
    df['image_path'] = df['FILE NAME'].apply(lambda x: f'{base_data_dir}/{category}/images/{x}.png')
    df['mask_path'] = df['FILE NAME'].apply(lambda x: f'{base_data_dir}/{category}/masks/{x}.png')
    df = df[df['image_path'].apply(os.path.exists)]  # Ensure file exists
    all_metadata = pd.concat([all_metadata, df], ignore_index=True)

# Data splitting
train_df, temp_df = train_test_split(all_metadata, test_size=0.4, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

# Data augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Data generator function with data augmentation and masks
def data_generator_segmentation(df, batch_size=32):
    while True:
        batch_paths = df.sample(n=batch_size)
        batch_input = []
        batch_output = []
        for _, row in batch_paths.iterrows():
            image = cv2.imread(row['image_path'], cv2.IMREAD_COLOR)
            mask = cv2.imread(row['mask_path'], cv2.IMREAD_GRAYSCALE)  # Load the mask
            if mask is not None:  # Ensure mask is loaded correctly
                image = cv2.resize(image, (256, 256))
                mask = cv2.resize(mask, (256, 256))  # Keep mask size to 256x256
                image = image / 255.0
                mask = mask / 255.0
                mask = np.round(mask).astype(np.float32)  # Ensure mask is binary and float32
                image = datagen.random_transform(image)
                mask = np.expand_dims(mask, axis=-1)  # Expand mask dimensions to (256, 256, 1)
                batch_input.append(image)
                batch_output.append(mask)  # Append mask directly
        batch_x = np.array(batch_input)
        batch_y = np.array(batch_output)
        yield batch_x, batch_y

# Setup data generators with data augmentation and masks
train_gen_segmentation = data_generator_segmentation(train_df, batch_size=32)
val_gen_segmentation = data_generator_segmentation(val_df, batch_size=32)
test_gen_segmentation = data_generator_segmentation(test_df, batch_size=32)

# Improved U-Net model with ResNet50 encoder and U-Net decoder maintaining 256x256 resolution
def build_unet_resnet50(input_shape):
    inputs = Input(shape=input_shape)

    # Encoder: ResNet50
    base_model = ResNet50(weights='imagenet', include_top=False, input_tensor=inputs)
    skip_connection_names = ['conv1_relu', 'conv2_block3_out', 'conv3_block4_out', 'conv4_block6_out']
    layers = [base_model.get_layer(name).output for name in skip_connection_names]

    # Decoder
    x = base_model.output
    num_filters = 256
    regularizer = l1_l2(l1=1e-5, l2=1e-4)
    for i in range(len(layers)):
        x = UpSampling2D()(x)
        x = Conv2D(num_filters // (2 ** i), (3, 3), activation='relu', padding='same', kernel_regularizer=regularizer)(x)
        x = concatenate([x, layers[-(i + 1)]])
        x = Conv2D(num_filters // (2 ** i), (3, 3), activation='relu', padding='same', kernel_regularizer=regularizer)(x)

    # Ensure final output matches the original input size
    x = UpSampling2D(size=(2, 2))(x)  # Up-sample to 256x256
    outputs = Conv2D(1, (1, 1), activation='sigmoid')(x)  # Use sigmoid activation for binary segmentation
    model = Model(inputs, outputs)
    return model

input_shape = (256, 256, 3)
unet_resnet50 = build_unet_resnet50(input_shape)

# Define the combined loss function
def dice_loss(y_true, y_pred):
    y_true_f = tf.keras.backend.flatten(y_true)
    y_pred_f = tf.keras.backend.flatten(y_pred)
    intersection = tf.keras.backend.sum(y_true_f * y_pred_f)
    return 1 - (2. * intersection + 1) / (tf.keras.backend.sum(y_true_f) + tf.keras.backend.sum(y_pred_f) + 1)

def combined_loss(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float32)  # Ensure y_true is float32
    y_pred = tf.cast(y_pred, tf.float32)  # Ensure y_pred is float32
    bce = tf.keras.losses.binary_crossentropy(y_true, y_pred)
    dice = dice_loss(y_true, y_pred)
    return bce + dice

unet_resnet50.compile(optimizer=Adam(learning_rate=0.0001), loss=combined_loss, metrics=['accuracy'])

# Callbacks for early stopping and learning rate reduction
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=1e-6)

# Model training with data augmentation and regularization
history_segmentation = unet_resnet50.fit(
    train_gen_segmentation,
    steps_per_epoch=len(train_df) // 32,
    validation_data=val_gen_segmentation,
    validation_steps=len(val_df) // 32,
    epochs=20,
    callbacks=[early_stopping, reduce_lr]
)

# Evaluate the model on the test set with augmentation
test_results_segmentation = unet_resnet50.evaluate(test_gen_segmentation, steps=len(test_df) // 32)
print(f"Test accuracy with segmentation model: {test_results_segmentation[1]*100:.2f}%")


Using GPU: PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


2024-06-17 23:39:10.854138: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-17 23:39:11.617495: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 78902 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-80GB, pci bus id: 0000:07:00.0, compute capability: 8.0


Epoch 1/20


2024-06-17 23:39:17.432212: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8201


205/205 [==============================] - 121s 555ms/step - loss: 1.5226 - accuracy: 0.8641 - val_loss: 2.1848 - val_accuracy: 0.7752 - lr: 1.0000e-04
Epoch 2/20
205/205 [==============================] - 113s 552ms/step - loss: 1.0913 - accuracy: 0.8884 - val_loss: 1.4469 - val_accuracy: 0.7987 - lr: 1.0000e-04
Epoch 3/20
205/205 [==============================] - 112s 550ms/step - loss: 0.8948 - accuracy: 0.8973 - val_loss: 1.0858 - val_accuracy: 0.8629 - lr: 1.0000e-04
Epoch 4/20
205/205 [==============================] - 112s 549ms/step - loss: 0.7742 - accuracy: 0.9038 - val_loss: 0.8808 - val_accuracy: 0.8692 - lr: 1.0000e-04
Epoch 5/20
205/205 [==============================] - 112s 549ms/step - loss: 0.6938 - accuracy: 0.9095 - val_loss: 0.9028 - val_accuracy: 0.8560 - lr: 1.0000e-04
Epoch 6/20
205/205 [==============================] - 112s 550ms/step - loss: 0.6360 - accuracy: 0.9141 - val_loss: 0.6965 - val_accuracy: 0.8966 - lr: 1.0000e-04
Epoch 7/20
205/205 [=============

In [1]:
import os
import numpy as np
import pandas as pd
import cv2
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, UpSampling2D, concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.model_selection import train_test_split

# TensorFlow GPU configuration
physical_devices = tf.config.list_physical_devices('GPU')
if len(physical_devices) > 0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
    print("Using GPU:", physical_devices[0])
else:
    print("No GPU found, using CPU")

# Set up paths and categories
base_data_dir = '/blue/srampazzi/vi.gade/cov/covid'
categories = ['COVID', 'Lung_Opacity', 'Normal', 'Viral_Pneumonia']

# Load metadata and preprocess
all_metadata = pd.DataFrame()
for category in categories:
    path = f'{base_data_dir}/{category}.metadata.xlsx'
    df = pd.read_excel(path, usecols=['FILE NAME', 'FORMAT', 'SIZE'])
    df['label'] = category
    df['image_path'] = df['FILE NAME'].apply(lambda x: f'{base_data_dir}/{category}/images/{x}.png')
    df['mask_path'] = df['FILE NAME'].apply(lambda x: f'{base_data_dir}/{category}/masks/{x}.png')
    df = df[df['image_path'].apply(os.path.exists)]  # Ensure file exists
    all_metadata = pd.concat([all_metadata, df], ignore_index=True)

# Data splitting
train_df, temp_df = train_test_split(all_metadata, test_size=0.4, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

# Data augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Data generator function with data augmentation and masks
def data_generator_segmentation(df, batch_size=32):
    while True:
        batch_paths = df.sample(n=batch_size)
        batch_input = []
        batch_output = []
        for _, row in batch_paths.iterrows():
            image = cv2.imread(row['image_path'], cv2.IMREAD_COLOR)
            mask = cv2.imread(row['mask_path'], cv2.IMREAD_GRAYSCALE)  # Load the mask
            if mask is not None:  # Ensure mask is loaded correctly
                image = cv2.resize(image, (256, 256))
                mask = cv2.resize(mask, (256, 256))  # Keep mask size to 256x256
                image = image / 255.0
                mask = mask / 255.0
                mask = np.round(mask).astype(np.float32)  # Ensure mask is binary and float32
                image = datagen.random_transform(image)
                mask = np.expand_dims(mask, axis=-1)  # Expand mask dimensions to (256, 256, 1)
                batch_input.append(image)
                batch_output.append(mask)  # Append mask directly
        batch_x = np.array(batch_input)
        batch_y = np.array(batch_output)
        yield batch_x, batch_y

# Setup data generators with data augmentation and masks
train_gen_segmentation = data_generator_segmentation(train_df, batch_size=32)
val_gen_segmentation = data_generator_segmentation(val_df, batch_size=32)
test_gen_segmentation = data_generator_segmentation(test_df, batch_size=32)

# Improved U-Net model with ResNet50 encoder and U-Net decoder maintaining 256x256 resolution
def build_unet_resnet50(input_shape):
    inputs = Input(shape=input_shape)

    # Encoder: ResNet50
    base_model = ResNet50(weights='imagenet', include_top=False, input_tensor=inputs)
    skip_connection_names = ['conv1_relu', 'conv2_block3_out', 'conv3_block4_out', 'conv4_block6_out']
    layers = [base_model.get_layer(name).output for name in skip_connection_names]

    # Decoder
    x = base_model.output
    num_filters = 256
    for i in range(len(layers)):
        x = UpSampling2D()(x)
        x = Conv2D(num_filters // (2 ** i), (3, 3), activation='relu', padding='same')(x)
        x = concatenate([x, layers[-(i + 1)]])
        x = Conv2D(num_filters // (2 ** i), (3, 3), activation='relu', padding='same')(x)

    # Ensure final output matches the original input size
    x = UpSampling2D(size=(2, 2))(x)  # Up-sample to 256x256
    outputs = Conv2D(1, (1, 1), activation='sigmoid')(x)  # Use sigmoid activation for binary segmentation
    model = Model(inputs, outputs)
    return model

input_shape = (256, 256, 3)
unet_resnet50 = build_unet_resnet50(input_shape)

# Define the combined loss function
def dice_loss(y_true, y_pred):
    y_true_f = tf.keras.backend.flatten(y_true)
    y_pred_f = tf.keras.backend.flatten(y_pred)
    intersection = tf.keras.backend.sum(y_true_f * y_pred_f)
    return 1 - (2. * intersection + 1) / (tf.keras.backend.sum(y_true_f) + tf.keras.backend.sum(y_pred_f) + 1)

def combined_loss(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float32)  # Ensure y_true is float32
    y_pred = tf.cast(y_pred, tf.float32)  # Ensure y_pred is float32
    bce = tf.keras.losses.binary_crossentropy(y_true, y_pred)
    dice = dice_loss(y_true, y_pred)
    return bce + dice

unet_resnet50.compile(optimizer=Adam(learning_rate=0.0001), loss=combined_loss, metrics=['accuracy'])

# Callbacks for early stopping and learning rate reduction
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=1e-6)

# Model training with data augmentation and regularization
history_segmentation = unet_resnet50.fit(
    train_gen_segmentation,
    steps_per_epoch=len(train_df) // 32,
    validation_data=val_gen_segmentation,
    validation_steps=len(val_df) // 32,
    epochs=20,
    callbacks=[early_stopping, reduce_lr]
)

# Evaluate the model on the test set with augmentation
test_results_segmentation = unet_resnet50.evaluate(test_gen_segmentation, steps=len(test_df) // 32)
print(f"Test accuracy with segmentation model: {test_results_segmentation[1]*100:.2f}%")


Using GPU: PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


2024-06-17 18:41:40.292346: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-17 18:41:41.106807: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 78911 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-80GB, pci bus id: 0000:87:00.0, compute capability: 8.0


Epoch 1/20


2024-06-17 18:41:46.553818: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8201


205/205 [==============================] - 127s 586ms/step - loss: 0.6818 - accuracy: 0.8632 - val_loss: 1.3180 - val_accuracy: 0.7742 - lr: 1.0000e-04
Epoch 2/20
205/205 [==============================] - 126s 616ms/step - loss: 0.5534 - accuracy: 0.8891 - val_loss: 0.8935 - val_accuracy: 0.8324 - lr: 1.0000e-04
Epoch 3/20
205/205 [==============================] - 126s 616ms/step - loss: 0.5084 - accuracy: 0.8990 - val_loss: 0.8455 - val_accuracy: 0.8329 - lr: 1.0000e-04
Epoch 4/20
205/205 [==============================] - 126s 616ms/step - loss: 0.4796 - accuracy: 0.9052 - val_loss: 0.7185 - val_accuracy: 0.8411 - lr: 1.0000e-04
Epoch 5/20
205/205 [==============================] - 124s 607ms/step - loss: 0.4429 - accuracy: 0.9125 - val_loss: 0.6573 - val_accuracy: 0.8631 - lr: 1.0000e-04
Epoch 6/20
205/205 [==============================] - 134s 654ms/step - loss: 0.4269 - accuracy: 0.9158 - val_loss: 0.5868 - val_accuracy: 0.8769 - lr: 1.0000e-04
Epoch 7/20
205/205 [=============

In [1]:
import os
import numpy as np
import pandas as pd
import cv2
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, UpSampling2D, concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.model_selection import train_test_split

# TensorFlow GPU configuration
physical_devices = tf.config.list_physical_devices('GPU')
if len(physical_devices) > 0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
    print("Using GPU:", physical_devices[0])
else:
    print("No GPU found, using CPU")

# Set up paths and categories
base_data_dir = '/blue/srampazzi/vi.gade/cov/covid'
categories = ['COVID', 'Lung_Opacity', 'Normal', 'Viral_Pneumonia']

# Load metadata and preprocess
all_metadata = pd.DataFrame()
for category in categories:
    path = f'{base_data_dir}/{category}.metadata.xlsx'
    df = pd.read_excel(path, usecols=['FILE NAME', 'FORMAT', 'SIZE'])
    df['label'] = category
    df['image_path'] = df['FILE NAME'].apply(lambda x: f'{base_data_dir}/{category}/images/{x}.png')
    df['mask_path'] = df['FILE NAME'].apply(lambda x: f'{base_data_dir}/{category}/masks/{x}.png')
    df = df[df['image_path'].apply(os.path.exists)]  # Ensure file exists
    all_metadata = pd.concat([all_metadata, df], ignore_index=True)

# Data splitting
train_df, temp_df = train_test_split(all_metadata, test_size=0.4, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

# Data augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Data generator function with data augmentation and masks
def data_generator_segmentation(df, batch_size=32):
    while True:
        batch_paths = df.sample(n=batch_size)
        batch_input = []
        batch_output = []
        for _, row in batch_paths.iterrows():
            image = cv2.imread(row['image_path'], cv2.IMREAD_COLOR)
            mask = cv2.imread(row['mask_path'], cv2.IMREAD_GRAYSCALE)  # Load the mask
            if mask is not None:  # Ensure mask is loaded correctly
                image = cv2.resize(image, (256, 256))
                mask = cv2.resize(mask, (128, 128))  # Resize mask to match output shape
                image = image / 255.0
                mask = mask / 255.0
                mask = np.round(mask).astype(np.float32)  # Ensure mask is binary and float32
                image = datagen.random_transform(image)
                batch_input.append(image)
                batch_output.append(mask.reshape(128, 128, 1))  # Reshape mask to match output dimensions
        batch_x = np.array(batch_input)
        batch_y = np.array(batch_output)
        yield batch_x, batch_y

# Setup data generators with data augmentation and masks
train_gen_segmentation = data_generator_segmentation(train_df, batch_size=32)
val_gen_segmentation = data_generator_segmentation(val_df, batch_size=32)
test_gen_segmentation = data_generator_segmentation(test_df, batch_size=32)

# Improved U-Net model with ResNet50 encoder and U-Net decoder
def build_unet_resnet50(input_shape):
    inputs = Input(shape=input_shape)

    # Encoder: ResNet50
    base_model = ResNet50(weights='imagenet', include_top=False, input_tensor=inputs)
    skip_connection_names = ['conv1_relu', 'conv2_block3_out', 'conv3_block4_out', 'conv4_block6_out']
    layers = [base_model.get_layer(name).output for name in skip_connection_names]

    # Decoder
    x = base_model.output
    num_filters = 256
    for i in range(len(layers)):
        x = UpSampling2D()(x)
        x = Conv2D(num_filters // (2 ** i), (3, 3), activation='relu', padding='same')(x)
        x = concatenate([x, layers[-(i + 1)]])
        x = Conv2D(num_filters // (2 ** i), (3, 3), activation='relu', padding='same')(x)

    outputs = Conv2D(1, (1, 1), activation='sigmoid')(x)  # Use sigmoid activation for binary segmentation
    model = Model(inputs, outputs)
    return model

input_shape = (256, 256, 3)
unet_resnet50 = build_unet_resnet50(input_shape)

# Define the combined loss function
def dice_loss(y_true, y_pred):
    y_true_f = tf.keras.backend.flatten(y_true)
    y_pred_f = tf.keras.backend.flatten(y_pred)
    intersection = tf.keras.backend.sum(y_true_f * y_pred_f)
    return 1 - (2. * intersection + 1) / (tf.keras.backend.sum(y_true_f) + tf.keras.backend.sum(y_pred_f) + 1)

def combined_loss(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float32)  # Ensure y_true is float32
    y_pred = tf.cast(y_pred, tf.float32)  # Ensure y_pred is float32
    print(f"combined_loss: y_true shape: {y_true.shape}, y_pred shape: {y_pred.shape}")  # Add debug prints
    bce = tf.keras.losses.binary_crossentropy(y_true, y_pred)
    dice = dice_loss(y_true, y_pred)
    return bce + dice

unet_resnet50.compile(optimizer=Adam(learning_rate=0.0001), loss=combined_loss, metrics=['accuracy'])

# Callbacks for early stopping and learning rate reduction
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=1e-6)

# Model training with data augmentation and regularization
history_segmentation = unet_resnet50.fit(
    train_gen_segmentation,
    steps_per_epoch=len(train_df) // 32,
    validation_data=val_gen_segmentation,
    validation_steps=len(val_df) // 32,
    epochs=20,
    callbacks=[early_stopping, reduce_lr]
)

# Evaluate the model on the test set with augmentation
test_results_segmentation = unet_resnet50.evaluate(test_gen_segmentation, steps=len(test_df) // 32)
print(f"Test accuracy with segmentation model: {test_results_segmentation[1]*100:.2f}%")


Using GPU: PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


2024-06-17 18:14:24.581476: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-17 18:14:25.375583: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 78911 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-80GB, pci bus id: 0000:87:00.0, compute capability: 8.0


Epoch 1/20
combined_loss: y_true shape: (None, None, None, None), y_pred shape: (None, 128, 128, 1)
combined_loss: y_true shape: (None, None, None, None), y_pred shape: (None, 128, 128, 1)


2024-06-17 18:14:30.892095: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8201


205/205 [==============================] - 136s 629ms/step - loss: 0.6645 - accuracy: 0.8612 - val_loss: 1.0314 - val_accuracy: 0.8137 - lr: 1.0000e-04
Epoch 2/20
205/205 [==============================] - 124s 610ms/step - loss: 0.5515 - accuracy: 0.8882 - val_loss: 0.9718 - val_accuracy: 0.8163 - lr: 1.0000e-04
Epoch 3/20
205/205 [==============================] - 120s 589ms/step - loss: 0.5182 - accuracy: 0.8953 - val_loss: 0.8182 - val_accuracy: 0.8011 - lr: 1.0000e-04
Epoch 4/20
205/205 [==============================] - 125s 614ms/step - loss: 0.4885 - accuracy: 0.9020 - val_loss: 0.7274 - val_accuracy: 0.8337 - lr: 1.0000e-04
Epoch 5/20
205/205 [==============================] - 128s 627ms/step - loss: 0.4541 - accuracy: 0.9092 - val_loss: 0.6393 - val_accuracy: 0.8586 - lr: 1.0000e-04
Epoch 6/20
205/205 [==============================] - 129s 633ms/step - loss: 0.4345 - accuracy: 0.9137 - val_loss: 0.5176 - val_accuracy: 0.8998 - lr: 1.0000e-04
Epoch 7/20
205/205 [=============

KeyboardInterrupt: 

In [1]:
import os
import numpy as np
import pandas as pd
import cv2
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.model_selection import train_test_split

# TensorFlow GPU configuration
physical_devices = tf.config.list_physical_devices('GPU')
if len(physical_devices) > 0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
    print("Using GPU:", physical_devices[0])
else:
    print("No GPU found, using CPU")

# Set up paths
base_data_dir = '/blue/srampazzi/vi.gade/cov/covid'
categories = ['COVID', 'Lung_Opacity', 'Normal', 'Viral_Pneumonia']

# Load metadata
all_metadata = pd.DataFrame()
for category in categories:
    path = f'{base_data_dir}/{category}.metadata.xlsx'
    df = pd.read_excel(path, usecols=['FILE NAME', 'FORMAT', 'SIZE'])
    df['label'] = category
    df['image_path'] = df['FILE NAME'].apply(lambda x: f'{base_data_dir}/{category}/images/{x}.png')
    df['mask_path'] = df['FILE NAME'].apply(lambda x: f'{base_data_dir}/{category}/masks/{x}.png')
    df = df[df['image_path'].apply(os.path.exists)]  # Ensure file exists
    all_metadata = pd.concat([all_metadata, df], ignore_index=True)

# Data splitting
train_df, temp_df = train_test_split(all_metadata, test_size=0.4, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

# Data augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Data generator function for segmentation
def data_generator_segmentation(df, batch_size=8):
    while True:
        batch_paths = df.sample(n=batch_size)
        batch_input = []
        batch_output = []
        for _, row in batch_paths.iterrows():
            image = cv2.imread(row['image_path'], cv2.IMREAD_COLOR)
            mask = cv2.imread(row['mask_path'], cv2.IMREAD_GRAYSCALE)  # Load the mask
            if mask is not None:  # Ensure mask is loaded correctly
                image = cv2.resize(image, (256, 256))
                mask = cv2.resize(mask, (256, 256))
                image = image / 255.0
                mask = mask / 255.0
                image = datagen.random_transform(image)
                mask = np.expand_dims(mask, axis=-1)  # Expand mask dimensions
                mask = datagen.random_transform(mask)
                batch_input.append(image)
                batch_output.append(mask)
        batch_x = np.array(batch_input)
        batch_y = np.array(batch_output)
        yield batch_x, batch_y

# Setup data generators
train_gen_segmentation = data_generator_segmentation(train_df, batch_size=8)
val_gen_segmentation = data_generator_segmentation(val_df, batch_size=8)
test_gen_segmentation = data_generator_segmentation(test_df, batch_size=8)

# Define U-Net model
def unet_model(input_size=(256, 256, 3)):
    inputs = Input(input_size)
    c1 = Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    c1 = Conv2D(64, (3, 3), activation='relu', padding='same')(c1)
    p1 = MaxPooling2D((2, 2))(c1)

    c2 = Conv2D(128, (3, 3), activation='relu', padding='same')(p1)
    c2 = Conv2D(128, (3, 3), activation='relu', padding='same')(c2)
    p2 = MaxPooling2D((2, 2))(c2)

    c3 = Conv2D(256, (3, 3), activation='relu', padding='same')(p2)
    c3 = Conv2D(256, (3, 3), activation='relu', padding='same')(c3)
    p3 = MaxPooling2D((2, 2))(c3)

    c4 = Conv2D(512, (3, 3), activation='relu', padding='same')(p3)
    c4 = Conv2D(512, (3, 3), activation='relu', padding='same')(c4)
    p4 = MaxPooling2D(pool_size=(2, 2))(c4)

    c5 = Conv2D(1024, (3, 3), activation='relu', padding='same')(p4)
    c5 = Conv2D(1024, (3, 3), activation='relu', padding='same')(c5)

    u6 = UpSampling2D(size=(2, 2))(c5)
    u6 = concatenate([u6, c4], axis=3)
    c6 = Conv2D(512, (3, 3), activation='relu', padding='same')(u6)
    c6 = Conv2D(512, (3, 3), activation='relu', padding='same')(c6)

    u7 = UpSampling2D(size=(2, 2))(c6)
    u7 = concatenate([u7, c3], axis=3)
    c7 = Conv2D(256, (3, 3), activation='relu', padding='same')(u7)
    c7 = Conv2D(256, (3, 3), activation='relu', padding='same')(c7)

    u8 = UpSampling2D(size=(2, 2))(c7)
    u8 = concatenate([u8, c2], axis=3)
    c8 = Conv2D(128, (3, 3), activation='relu', padding='same')(u8)
    c8 = Conv2D(128, (3, 3), activation='relu', padding='same')(c8)

    u9 = UpSampling2D(size=(2, 2))(c8)
    u9 = concatenate([u9, c1], axis=3)
    c9 = Conv2D(64, (3, 3), activation='relu', padding='same')(u9)
    c9 = Conv2D(64, (3, 3), activation='relu', padding='same')(c9)

    outputs = Conv2D(1, (1, 1), activation='sigmoid')(c9)

    model = Model(inputs=[inputs], outputs=[outputs])

    return model

# Build and compile the U-Net model
unet = unet_model()
unet.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

# Callbacks for early stopping and learning rate reduction
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.00001)

# Model training with data augmentation and regularization
history_segmentation = unet.fit(
    train_gen_segmentation,
    steps_per_epoch=len(train_df) // 8,
    validation_data=val_gen_segmentation,
    validation_steps=len(val_df) // 8,
    epochs=20,
    callbacks=[early_stopping, reduce_lr]
)

# Evaluate the model on the test set with augmentation
test_results_segmentation = unet.evaluate(test_gen_segmentation, steps=len(test_df) // 8)
print(f"Test accuracy with augmentation and regularization: {test_results_segmentation[1]*100:.2f}%")


Using GPU: PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


2024-06-16 23:59:50.564483: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-16 23:59:51.365159: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 78911 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-80GB, pci bus id: 0000:07:00.0, compute capability: 8.0


Epoch 1/20


2024-06-16 23:59:52.965022: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8201


822/822 [==============================] - 146s 172ms/step - loss: 0.4727 - accuracy: 0.7735 - val_loss: 0.4605 - val_accuracy: 0.7749 - lr: 1.0000e-04
Epoch 2/20
822/822 [==============================] - 142s 172ms/step - loss: 0.4624 - accuracy: 0.7738 - val_loss: 0.4616 - val_accuracy: 0.7727 - lr: 1.0000e-04
Epoch 3/20
822/822 [==============================] - 141s 171ms/step - loss: 0.4614 - accuracy: 0.7735 - val_loss: 0.4584 - val_accuracy: 0.7744 - lr: 1.0000e-04
Epoch 4/20
822/822 [==============================] - 140s 171ms/step - loss: 0.4616 - accuracy: 0.7729 - val_loss: 0.4643 - val_accuracy: 0.7721 - lr: 1.0000e-04
Epoch 5/20
822/822 [==============================] - 140s 171ms/step - loss: 0.4616 - accuracy: 0.7732 - val_loss: 0.4592 - val_accuracy: 0.7717 - lr: 1.0000e-04
Epoch 6/20
822/822 [==============================] - 140s 170ms/step - loss: 0.4605 - accuracy: 0.7731 - val_loss: 0.4580 - val_accuracy: 0.7732 - lr: 1.0000e-04
Epoch 7/20
822/822 [=============